<img src=https://image.shutterstock.com/image-photo/kazan-russian-federation-aug-5-600w-1211046340.jpg>


### 1.Introduccion

Se proporcionan datos históricos de ventas para 45 tiendas Walmart ubicadas en diferentes regiones. Cada tienda contiene varios departamentos y se tiene la tarea de predecir las ventas de todo el departamento para cada tienda.
Además, Walmart organiza varios eventos promocionales de rebajas durante todo el año. Estas rebajas preceden a los feriados importantes, los cuatro más grandes de los cuales son el Super Bowl, el Labor Day, Thanksgiving y Christmas. Las semanas que incluyen estos días festivos se ponderan cinco veces más en la evaluación que las semanas que no son festivos. Parte del desafío que presenta esta competencia es modelar los efectos de las rebajas en estas semanas de vacaciones en ausencia de datos históricos completos / ideales



### 2. Importar Librerías & Cargar datos

###  2.1 Librerías a importar

In [1]:
!pip install -U notebook-as-pdf
!pip install pyppeteer


!pip install plotly
!pip install pingouin
!pip install missingno
!pip install xgboost lightgbm --upgrade --quiet
!pip install tpot


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-whitegrid')
from sklearn import metrics

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from datetime import datetime
import math
import pingouin as pg
import plotly.express as px
import plotly.graph_objs as go
import warnings
warnings.filterwarnings('ignore')

 


from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\cesar\\anaconda3\\Lib\\site-packages\\~-boost\\lib\\xgboost.dll'
Consider using the `--user` option or check the permissions.



### 2.2 Carga de datos

In [2]:
features = pd.read_csv("features.csv")
sampleSubmission = pd.read_csv("sampleSubmission.csv")
stores = pd.read_csv("stores.csv")
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")


### 2.3 Inspección inicial
Observamos el tipo de dato de cada una de las variables para los diferentes archivos:



In [3]:
pd.DataFrame(features.dtypes, columns=['Type']).T

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
Type,int64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool


In [4]:
pd.DataFrame(stores.dtypes, columns=['Type']).T

,Store,Type,Size
Type,int64,object,int64


In [5]:
pd.DataFrame(train.dtypes, columns=['Type']).T

,Store,Dept,Date,Weekly_Sales,IsHoliday
Type,int64,int64,object,float64,bool


In [6]:
pd.DataFrame(test.dtypes, columns=['Type']).T

,Store,Dept,Date,IsHoliday
Type,int64,int64,object,bool


Evaluamos la cantidad de filas y columnas en cada uno de los datasets. Observamos que train y test comprenden el 78% y 22% respectivamente del total de los datos de las 45 tiendas.

In [7]:
print(features.shape)
print(stores.shape)
print(train.shape)
print(test.shape)
print("El ratio de train data : test data es ", 
      (round(train.shape[0]*100/(train.shape[0]+test.shape[0])),100-round(train.shape[0]*100/(train.shape[0]+test.shape[0]))))

(8190, 12)
(45, 3)
(421570, 5)
(115064, 4)
El ratio de train data : test data es  (79, 21)


¿Qué información hemos obtenido hasta ahora?

* El dataset _'features', 'train' y 'test'_ poseen la fecha en formato objet y debe ser modificada.
* El dataset 'stores' posee 45 filas correspondientes al número de tiendas que vamos a analizar
* Los conjuntos _'train'_ y _'test_ comprenden el 78% y 22% respectivamente del total de los datos de estudio.


###  2.4 Merge de datasets 'features' y 'stores'

Vamos a crear un nuevo dataset en el que a 'Features' le vamos a incluir el área de la tienda, así como su tipo que 
aparecen en 'Stores' utilizando como elemento común entre ambos la columna 'Store' siendo la unión a través de inner join.
Comprobamos que el nuevo dataset "feature_store" posee las 8190 filas, pero con 14 variables en vez de 12 para las 45 tiendas.




In [8]:
feature_store = features.merge(stores, how='inner', on = "Store")

In [9]:
print("El numero de filas y columnas es de: ",(feature_store.shape))
print("Existen",(len(feature_store.Store.unique())), 'tiendas unicas')

El numero de filas y columnas es de:  (8190, 14)
Existen 45 tiendas unicas


### 2.5 Merge de datasets 'train' & 'feature_store'

Realizamos la operación para los dataset 'train' & 'feature_store'.

In [10]:

train_df = train.merge(feature_store, how='inner', 
                       on = ['Store','Date','IsHoliday']).sort_values(by=['Store','Dept','Date']).reset_index(drop=True)

In [11]:
print("El numero de filas y columnas es de: ",(train_df.shape))

El numero de filas y columnas es de:  (421570, 16)


### 2.6 Merge de datasets 'test' & 'feature_store'

Realizamos la misma operación para los dataset 'test' & 'feature_store'. El nuevo dataset mantiene las mismas filas
que 'test' y tiene las mismas variables que 'train’ a excepción de 'Weekly_Sales'



In [12]:
test_df = test.merge(feature_store, how='inner', 
                     on = ['Store','Date','IsHoliday']).sort_values(by = ['Store','Dept','Date']).reset_index(drop=True)

In [13]:
print("El numero de filas y columnas es de: ",(test_df.shape))

El numero de filas y columnas es de:  (115064, 15)


### 2.7 Convertir columna "Date" a formato datetime



In [14]:
train_df['Date'] = pd.to_datetime(train_df['Date'])
test_df['Date'] = pd.to_datetime(test_df['Date'])

print(train_df["Date"].dtypes)
print(test_df["Date"].dtypes)

datetime64[ns]
datetime64[ns]


### 2.8 Descomponemos la fecha creando nuevas variables 'Day','Week','Month','Year'

In [15]:

train_df['Day'] = train_df['Date'].dt.day
train_df['Week'] = train_df['Date'].dt.week
train_df['Month'] = train_df['Date'].dt.month
train_df['Year'] = train_df['Date'].dt.year


test_df['Day'] = test_df['Date'].dt.day
test_df['Week'] = test_df['Date'].dt.week
test_df['Month'] = test_df['Date'].dt.month
test_df['Year'] = test_df['Date'].dt.year

print("El numero de filas y columnas es de: ",(train_df.shape))
print("El numero de filas y columnas es de: ",(test_df.shape))


El numero de filas y columnas es de:  (421570, 20)
El numero de filas y columnas es de:  (115064, 19)


In [16]:
train_df.head(1)

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,Day,Week,Month,Year
0,1,1,2010-02-05,24924.5,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315,5,5,2,2010


In [17]:
test_df.head(1)

,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,Day,Week,Month,Year
0,1,1,2012-11-02,False,55.32,3.386,6766.44,5147.7,50.82,3639.9,2737.42,223.462779,6.573,A,151315,2,44,11,2012


### 4. Feature Engineering


In [18]:
data_train = train_df.copy()
data_test = test_df.copy()

### 4.1 Feature Engineering Nueva variable:  Semanas Clave


Se van a crear cuatro nuevas variables que corresponden a aquellas semanas que se consideran claves en las ventas, estas serían:

* SuperBowlWeek: Semana 6 del año.
* LaborDay: Semana 36 del año.
* Thanksgiving: Semana 47 del año.
* Christmas: Semana 52 del año.




In [19]:
data_train['SuperBowlWeek'] = train_df['Week'].apply(lambda x: 1 if x == 6 else 0)
data_train['LaborDay'] = train_df['Week'].apply(lambda x: 1 if x == 36 else 0)
data_train['Thanksgiving'] = train_df['Week'].apply(lambda x: 1 if x == 47 else 0)
data_train['Christmas'] = train_df['Week'].apply(lambda x: 1 if x == 52 else 0)

In [20]:
data_test['SuperBowlWeek'] = test_df['Week'].apply(lambda x: 1 if x == 6 else 0)
data_test['LaborDay'] = test_df['Week'].apply(lambda x: 1 if x == 36 else 0)
data_test['Thanksgiving'] = test_df['Week'].apply(lambda x: 1 if x == 47 else 0)
data_test['Christmas'] = test_df['Week'].apply(lambda x: 1 if x == 52 else 0)

###  4.2 Feature Engineering nueva variable: Quarter


Agrupamos las semanas por trimestre Q1, Q2,Q3 y Q4. El motivo es para conocer si las ventas y las promociones estan sujetas a los resultados trimestrales que pueda ofrecer la empresa.

* Q1:Semana de la 1 a la 13.
* Q2:Semana de la 14 a la 26.
* Q3:Semana de la 27 a la 39.
* Q4:Semana de la 40 a la 52.


In [21]:
data_train.insert(24,'Quarter',data_train['Week'])
data_test.insert(23,'Quarter',data_test['Week'])

In [22]:
data_train['Quarter'] = data_train['Quarter'].replace([1,2,3,4,5,6,7,8,9,10,11,12,13], 1)
data_train['Quarter'] = data_train['Quarter'].replace([14,15,16,17,18,19,20,21,22,23,24,25,26], 2)
data_train['Quarter'] = data_train['Quarter'].replace([27,28,29,30,31,32,33,34,35,36,37,38,39], 3)
data_train['Quarter'] = data_train['Quarter'].replace([40,41,42,43,44,45,46,47,48,49,50,51,52], 4)

data_test['Quarter'] = data_train['Quarter'].replace([1,2,3,4,5,6,7,8,9,10,11,12,13], 1)
data_test['Quarter'] = data_train['Quarter'].replace([14,15,16,17,18,19,20,21,22,23,24,25,26], 2)
data_test['Quarter'] = data_train['Quarter'].replace([27,28,29,30,31,32,33,34,35,36,37,38,39], 3)
data_test['Quarter'] = data_train['Quarter'].replace([40,41,42,43,44,45,46,47,48,49,50,51,52], 4)


### 4.3 Feature Engineering nueva variable: Rebajas


Creamos una nueva variable 'MarkdownSum' que será el sumatorio de las cuatro promociones Markdown. En caso de que 2 promociones se solapen en el tiempo se verá reflejado en el sumatorio.



In [23]:
data_train['MarkdownsSum'] = train_df['MarkDown1'] + train_df['MarkDown2'] + train_df['MarkDown3'] + train_df['MarkDown4'] + train_df['MarkDown5']
data_test['MarkdownsSum'] = test_df['MarkDown1'] + test_df['MarkDown2'] + test_df['MarkDown3'] + test_df['MarkDown4'] + test_df['MarkDown5']

### 4.4 Feature Engineering: Rellenar valores missing

Podemos comprobar en los gráficos adjuntos la ausencia de bastantes valores en las variables de las rebajas, CPI y Unemployment.

Las rebajas cuando sean NA el valor será sustituido por 0, CPI y Unemployment serán rellenos por la media.


#### 4.4.1 Observamos los valores faltantes

In [24]:
#Package called missingno (https://github.com/ResidentMario/missingno) !pip install quilt
import missingno as msno
msno.matrix(data_train)

MemoryError: Unable to allocate 125. MiB for an array with shape (421570, 26, 3) and data type float32

In [ ]:
msno.matrix(data_test)

In [ ]:
# Sumatorio de valores missing para el dataset 'train'.
print('Datos missing en data_train')
print('---------------------')
print(data_train.isna().sum())
print('')
print('Datos missing en data_test')
print('---------------------')
print(data_test.isna().sum())

#### 4.4.2 Los valores missing en 'data_train' los rellenamos con el valor '0'.

Para el caso de 'data_train' solo se observan valores missing en las variables Markdown.

In [ ]:
data_train.fillna(0, inplace = True)

#### 4.4.3 Los valores missing en 'data_test' correspondientes a 'CPI' y 'Unemployment' los rellenamos con la media.


In [ ]:
data_test['CPI'].fillna(data_test['CPI'].mean(), inplace = True)
data_test['Unemployment'].fillna(data_test['Unemployment'].mean(), inplace = True)

In [ ]:
# Los valores missing en 'test' que corresponden solamente a las rebajas los rellenamos con "0"
data_test.fillna(0, inplace = True)

### 4.5 Feature Engineering: IsHoliday Encoding Categorical Data

La variable 'IsHoliday' tendrá valor 1 si su valor original es 'True' y 'False' en caso contrario.



In [ ]:
data_train['IsHoliday'] = data_train['IsHoliday'].apply(lambda x: 1 if x == True else 0)
data_test['IsHoliday'] = data_test['IsHoliday'].apply(lambda x: 1 if x == True else 0)

###  4.6 Feature Engineering : Type  Encoding Categorical Data.

la variable 'Type' tendra valor 1 si la tienda era de la categoria A, 2 si es de la categoria B y 3 si es de la categoria C.

In [ ]:
data_train['Type'] = data_train['Type'].apply(lambda x: 1 if x == 'A' else (2 if x == 'B' else 3))
data_test['Type'] = data_test['Type'].apply(lambda x: 1 if x == 'A' else (2 if x == 'B' else 3))

### 4.7 Feature importance.

#### Selección de variables

Una vez rellenos los valores missing observamos que las variables que poseen más correlación con las ventas semanales son el tamaño de la tienda, el departamento, así como las diferentes campañas de rebajas.


In [ ]:
data_train.corr()['Weekly_Sales'][:5].sort_values(ascending = False)

In [ ]:
#Observamos el tipo de dato de cada una de las variables  para los diferentes archivos:
pd.DataFrame(data_train.dtypes, columns=['Type']).T

In [ ]:
X_f = data_train.drop(['Date','Weekly_Sales'], axis = 'columns' )
y_f = data_train['Weekly_Sales']

In [ ]:

rf_features = RandomForestRegressor() 

In [ ]:
%%time
rf_features.fit(X_f, y_f)

In [ ]:
importance_df = pd.DataFrame({
    'feature': X_f.columns,
    'importance': rf_features.feature_importances_
}).sort_values('importance', ascending=False)

In [ ]:
plt.figure(figsize=(10,6))
plt.title('Feature Importance')
sns.barplot(data=importance_df.head(26), x='importance', y='feature');

In [ ]:
data_train.dtypes

In [ ]:
dt_vif = data_train.copy(deep = True)
features = list(dt_vif.columns)
features.remove('Weekly_Sales') # Borrado de la variable objetivo
features.remove('Date') # Una variable en formato Date no puede ser procesada por la funcion.
dt_vif = dt_vif[features]

for i in range(len(features)):
    var = features[i]
    fet = features[:]
    fet.remove(var)
    
    x = dt_vif[fet]
    y = data_train[var]
    
    model = LinearRegression()
    model.fit(x, y)
    
    vif = 1 / (1 - model.score(x, y))
    
    print ('El valor del VIF para la variable', var, 'es:', vif)

Se observan algunas variables que tienen un VIF superior a 5 y algunos valores son muy elevados, procederemos a irlas eliminando y ejecutaremos el proceso nuevamente

In [ ]:

dt_vif = data_train.copy(deep = True)
features = list(dt_vif.columns)
features.remove('Weekly_Sales')
features.remove('Date')
features.remove('MarkdownsSum') # Eliminamos esta variable por ser la que posee mas VIF del conjunto Markdown
features.remove('Week') # Eliminamos la variable Week por tener un VIF de 40,090
dt_vif = dt_vif[features]

for i in range(len(features)):
    var = features[i]
    fet = features[:]
    fet.remove(var)
    
    x = dt_vif[fet]
    y = data_train[var]
    
    model = LinearRegression()
    model.fit(x, y)
    
    vif = 1 / (1 - model.score(x, y))
    
    print ('El valor del VIF para la variable', var, 'es:', vif)

In [ ]:
dt_vif = data_train.copy(deep = True)
features = list(dt_vif.columns)
features.remove('Weekly_Sales')
features.remove('Date')
features.remove('MarkdownsSum') 
features.remove('Week') 
features.remove('Quarter') # Esta variable es la siguientye candidata por tener un VIF de 15


dt_vif = dt_vif[features]

for i in range(len(features)):
    var = features[i]
    fet = features[:]
    fet.remove(var)
    
    x = dt_vif[fet]
    y = data_train[var]
    
    model = LinearRegression()
    model.fit(x, y)
    
    vif = 1 / (1 - model.score(x, y))
    
    print ('El valor del VIF para la variable', var, 'es:', vif)


Existen algunas variables que presentan valor infinito por tanto eliminamos la siguiente variable
   * Eliminar la variable IsHoliday
   

In [ ]:
dt_vif = data_train.copy(deep = True)
features = list(dt_vif.columns)
features.remove('Weekly_Sales')
features.remove('Date')
features.remove('MarkdownsSum') 
features.remove('Week') 
features.remove('Quarter') 
features.remove('IsHoliday') # Eliminamos IsHoliday por tener valor Inf.


dt_vif = dt_vif[features]

for i in range(len(features)):
    var = features[i]
    fet = features[:]
    fet.remove(var)
    
    x = dt_vif[fet]
    y = data_train[var]
    
    model = LinearRegression()
    model.fit(x, y)
    
    vif = 1 / (1 - model.score(x, y))
    
    print ('El valor del VIF para la variable', var, 'es:', vif)


### Guardamos los dataset

In [ ]:
#Dataset SIN feature engineering
train_df.to_csv('train_df.csv',index=False)
test_df.to_csv('test_df.csv',index=False)


#Dataset CON feature engineering
data_train.to_csv('data_train.csv',index=False)
data_test.to_csv('data_test.csv',index=False)
